# Preprocessing

## Steps

### - [user_setting_mapping](#user_setting_mapping)

### - [trips_users_df](#trips_users_df)

1. Read all data until 30-10-2019
2. Read updated data until 16-12-2019
3. Merge and create `trips_users_df`

### - [users_df_with_trips](#users_df_with_trips)

1. Read `usersData_normalized` for each campain that contains data about **all** users (updated to 16-12-2019)
2. Remove users without trips
3. Add home_address and work_address
4. Preprocess home and work address **TODO**
5. Extract generic worthwhileness values
6. Extract specific worthwhileness values
7. Solve language problem
8. Save `users_df_with_trips`

### - [trips_df](#trips_df)

1. Read all data until 30-10-2019
2. Read updated data until 16-12-2019
3. Merge and create `trips_df`
4. Take only trips also in `trips_users_df`
5. Preprocessing on dates
6. Save
7. Extract objectives

### - [all_legs](#all_legs)

1. Extract legs from `trips_df`
2. Preprocessing on legs
3. Add category transport
4. Add leg duration and preprocessing on duration and distance


### - [all_legs_final_ds_user_info](#all_legs_final_ds_user_info)

1. Merge legs info with users info and trip info

### - [all_legs_merged](#all_legs_merged)

1. Merge legs
2. Average Wasted Time for merged legs

### - [all_factors](#all_factors)

1. Select factors from each leg
2. Select generic activities from each leg
3. Create the Encodings
4. Map the encoded factors
5. Check not encoded factors

### - [values_from_trip](#values_from_trip)

1. Extract values from trips

### - [outliers_detection](#outliers_detection)


## Savings

- `trips_users_df.pkl`
- `all_users_df.pkl`
- `gen_worthwhile.pkl`
- `specific_worthwhile.pkl`
- `mot_text.txt`
- `mot_motText.txt`
- `users_df_with_trips.pkl`
- `trips_df_original.pkl`
- `trip_objs.pkl`
- `original_all_legs.pkl`
- `all_legs.pkl`
- `category_transp_mode_dict.json`
- `all_legs_final_ds_user_info.pkl`
- `all_legs_merged.pkl`
- `merged_legs.json`
- `all_legs_merged_1.pkl`
- `all_factors_original.pkl`
- `all_generic_activities.pkl`
- `all_factors.pkl`
- `values_from_trip.pkl`
- `all_legs_merged_no_outlier_0.01.pkl`

In [ ]:
# Import libraries

import os
import sys
import glob
import json
import time
import warnings
from datetime import date, datetime

import numpy as np
import pandas as pd
import calendar
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style"))

%matplotlib inline

In [ ]:
# Global variables
CUTTING_DATE = "2019-05-01"  # remove trips and data published before this date

meta_data_path = "../../data-campaigns/meta-data/"

# ========= ========= ========= ========= =========
input_path = "../../data-campaigns/2019-10-30.all/"
update_path = "../../data-campaigns/2019-12-16.update/"
out_path = "../../2019-12-16.out/"

# anon dataset (output)
anon_dataset_dir = out_path.rstrip("/").split("/")[-1].replace(".out", ".anon")
anon_dataset_path = os.path.join("../..", "anon-dataset", anon_dataset_dir)
# ========= ========= ========= ========= =========

In [ ]:
try:
    os.makedirs(os.path.abspath(out_path))
except FileExistsError:
    print("Directory '{}' already exists".format(out_path), file=sys.stderr)

try:
    os.makedirs(os.path.abspath(anon_dataset_path))
except FileExistsError:
    print("Directory '{}' already exists".format(anon_dataset_path), file=sys.stderr)

## Read data

<a id='user_setting_mapping'></a>
### user_setting_mapping

In [ ]:
user_setting_mapping = pd.read_csv(
    meta_data_path + "user-settings-values-mapping-unfolded_original.csv",
    encoding="cp1252",
    header=None,
)
user_setting_mapping.columns = ["dbvalues", "value"]
user_setting_mapping = user_setting_mapping.dropna()

user_setting_mapping_dict = {}
for ix, row in user_setting_mapping.iterrows():
    user_setting_mapping_dict[row["dbvalues"]] = row["value"]

# user_setting_mapping_dict

<a id='trips_users_df'></a>
### trips_users_df

- Read all data until 30-10-2019
- Read updated data until 16-12-2019
- Merge and create `trips_users_df`

**1. All data until 30-10-2019**

In [ ]:
## 1. All data

trips_prev_users_df = None
tempdf = None

read_files = glob.glob(input_path + "*_tripsUsersRelationData.json")
print(read_files)

count_trips = 0
for f in read_files:
    print(f)
    with open(f) as f:
        trips_users = json.loads(f.read())
        tempdf = json_normalize(trips_users)

    print(tempdf.shape)
    count_trips += tempdf.shape[0]

    if trips_prev_users_df is None:
        trips_prev_users_df = tempdf
    else:
        trips_prev_users_df = pd.concat([trips_prev_users_df, tempdf])

print("Trips = ", count_trips)
print(trips_prev_users_df.shape)

print("Remove duplicates...")
trips_prev_users_df = trips_prev_users_df.drop_duplicates(keep="first")
print(trips_prev_users_df.shape)
trips_prev_users_df["startDate_formated"] = pd.to_datetime(
    trips_prev_users_df["startDate"], unit="ms"
)
trips_prev_users_df = trips_prev_users_df[
    trips_prev_users_df["startDate_formated"] >= CUTTING_DATE
]
print("total trips: ", trips_prev_users_df.shape[0])
print("total unique trips: ", len(trips_prev_users_df["tripid"].unique()))
trips_prev_users_df["startDateDay"] = trips_prev_users_df["startDate_formated"].dt.date

**2. Updated data**

In [ ]:
## 2. Updated data

temp = None
trips_update_users_df = None

read_files = glob.glob(update_path + "*_tripsUsersRelationData.json")
print(read_files)

count_trips_update = 0
for f in read_files:
    print(f)
    with open(f) as f:
        trips_users = json.loads(f.read())
        temp = json_normalize(trips_users)

    print(temp.shape)
    count_trips_update += temp.shape[0]

    if trips_update_users_df is None:
        trips_update_users_df = temp
    else:
        trips_update_users_df = pd.concat([trips_update_users_df, temp])

print("Trips (update) = ", count_trips_update)
print(trips_update_users_df.shape)

print("Remove duplicates...")
trips_update_users_df = trips_update_users_df.drop_duplicates(keep="first")
print(trips_update_users_df.shape)
trips_update_users_df["startDate_formated"] = pd.to_datetime(
    trips_update_users_df["startDate"], unit="ms"
)
trips_update_users_df = trips_update_users_df[
    trips_update_users_df["startDate_formated"] >= CUTTING_DATE
]
print("total trips: ", trips_update_users_df.shape[0])
print("total unique trips: ", len(trips_update_users_df["tripid"].unique()))
trips_update_users_df["startDateDay"] = trips_update_users_df[
    "startDate_formated"
].dt.date

In [ ]:
## check
print("trips DF shape: {}".format(trips_prev_users_df.shape))
print("trips_update DF shape: {}".format(trips_update_users_df.shape))

**3. Merge all + updated**

In [ ]:
## 3. Merge all + updated
trips_users_df = pd.concat([trips_prev_users_df, trips_update_users_df])
print("trips DF shape: {}".format(trips_users_df.shape))

print("Remove duplicates...")
trips_users_df = trips_users_df.drop_duplicates(keep="first")
print("trips DF shape: {}".format(trips_users_df.shape))
print()
print("Total trips: ", trips_users_df.shape[0])
print("Total unique trips: ", len(trips_users_df["tripid"].unique()))
print("Total number of unique users: ", len(trips_users_df["userid"].unique()))

## SAVE
trips_users_df.to_pickle(out_path + "trips_users_df.pkl")
trips_users_df.head(3)

<a id='users_df_with_trips'></a>
### users_df_with_trips

1. Read `usersData_normalized` for each campain that contains data about **all** users (updated to 16-12-2019)
2. Remove users without trips
3. Add home_address and work_address
4. Preprocess home and work address **TODO**
5. Extract generic worthwhileness values
6. Extract specific worthwhileness values
7. Solve language problem
8. Save

**1. Read *all* users**

In [ ]:
### 1. Read *all* users
users_df = []
first = True
read_files = glob.glob(input_path + "*_usersData_normalized.json")
print(read_files)

tot = 0
for f in read_files:
    print(f)
    with open(f) as f:
        users = json.loads(f.read())
        users_df_temp = json_normalize(users)
    print(users_df_temp.shape)
    tot += users_df_temp.shape[0]

    if first:
        users_df = users_df_temp
        first = False
    else:
        users_df = pd.concat([users_df, users_df_temp])

print()
print("Tot = ", tot)
print(users_df.shape)

print()
print("Remove duplicates...")
users_df = users_df.drop_duplicates(["userid"], keep="first")
print(users_df.shape)

### SAVE all users
users_df.to_pickle(out_path + "all_users_df.pkl")

**2. Remove users without trips**

In [ ]:
### 2. Remove users without trips
print()
print(
    "Remove users that do not perform at least 1 trip i.e. remove users without trips"
)
users_df_with_trips = users_df[
    users_df.userid.isin(list(trips_users_df.userid.unique()))
]
### Rename columns
users_df_with_trips.columns = [
    w.replace("userSettings.", "") for w in users_df_with_trips.columns
]

print("- Unique users:", users_df["userid"].unique().shape[0])
print("- Users with at least 1 trip:", users_df_with_trips.shape[0])

users_with_notrips = users_df[
    ~users_df.userid.isin(list(trips_users_df.userid.unique()))
]
print("- Users with no trips:", users_with_notrips.shape[0])

**3. Add homeAddress and workAddress**

In [ ]:
# Read data
users_df_address = pd.DataFrame()
first = True
file_path = meta_data_path + "user-data-w-addresses/"
read_files = glob.glob(file_path + "*_Global_usersData.json")
print(read_files)

tot = 0
for f in read_files:
    print(f)
    with open(f) as f:
        users = json.loads(f.read())
        users_df_temp = json_normalize(users)
    print(users_df_temp.shape)
    tot += users_df_temp.shape[0]

    if first:
        users_df_address = users_df_temp
        first = False
    else:
        users_df_address = pd.concat([users_df_address, users_df_temp])

print()
print(users_df_address.shape)
print("Remove duplicates...")
users_df_address = users_df_address.drop_duplicates(["userid"], keep="first")
print(users_df_address.shape)
print("unique users:", len(users_df_address.userid.unique()))

### Rename columns
users_df_address.columns = [
    w.replace("userSettings.", "") for w in users_df_address.columns
]

### Add the address to users_df_with_trips
print()
print("add home and work address to users_df_with_trips..")
users_df_with_trips = pd.merge(
    users_df_with_trips,
    users_df_address[
        [
            "userid",
            "workAddress._id",
            "workAddress.address",
            "homeAddress._id",
            "homeAddress.address",
        ]
    ],
    on="userid",
    how="left",
)

print(users_df_with_trips.shape)
print(
    "users with home address",
    len(users_df_with_trips[~users_df_with_trips["homeAddress.address"].isna()]),
)
print(
    "users with work address",
    len(users_df_with_trips[~users_df_with_trips["workAddress.address"].isna()]),
)

**4. preprocess home and work**

**5. Extract generic worthwhileness values**

In [ ]:
gen_worthwhile = users_df_with_trips[["userid", "actValue", "prodValue", "relValue"]]
gen_worthwhile.columns = ["userid", "fitness", "productivity", "enjoyment"]

## Save
gen_worthwhile.to_pickle(out_path + "gen_worthwhile.pkl")

print("users with generic WW:", len(gen_worthwhile.userid.unique()))
gen_worthwhile.head(3)

**5. (b) Extract generic worthwhileness values**

In [ ]:
user_generic_worthwhileness_table = gen_worthwhile.copy()

In [ ]:
colstorename = {
    "userid": "userid",
    "fitness": "fit",
    "productivity": "prod",
    "enjoyment": "enjoy",
}

columnstokeep = [col for col in colstorename.keys()]
csv_columns = [col for col in colstorename.values()]

In [ ]:
user_generic_worthwhileness_table.rename(columns=colstorename, inplace=True)
print(user_generic_worthwhileness_table.shape)

In [ ]:
colstypes = {
    "userid": "str",
    "fit": "int",
    "prod": "int",
    "enjoy": "int",
}

user_generic_worthwhileness_table = user_generic_worthwhileness_table.astype(colstypes)
user_generic_worthwhileness_table.head(3)

In [ ]:
## save to csv
output_file = "user_generic_worthwhileness_values.csv"
output_path = os.path.join(anon_dataset_path, output_file)

user_generic_worthwhileness_table.to_csv(
    output_path, index=False, header=True, columns=csv_columns
)

**6. Extract specific worthwhileness values**

In [ ]:
specific_worthwhile = []
first = True
for index, user in users_df_with_trips.iterrows():

    specific_worthwhile = pd.DataFrame(list(user["preferedMots"]))
    specific_worthwhile["userid"] = user["userid"]
    if first:
        all_users_pref_mot = specific_worthwhile
        first = False
    else:
        all_users_pref_mot = pd.concat(
            [all_users_pref_mot, specific_worthwhile], sort=True
        )

specific_worthwhile.columns = [
    "Mot",
    "MotText",
    "_id",
    "fitness",
    "productivity",
    "enjoyment",
    "userid",
]

## Save
all_users_pref_mot.to_pickle(out_path + "specific_worthwhile.pkl")

print("users with specific WW:", len(all_users_pref_mot.userid.unique()))
all_users_pref_mot.head(3)

In [ ]:
### Save different motText and association mot - motText

# mot_text
mottext_lst = list(all_users_pref_mot.MotText.unique())
with open(out_path + "mot_text.txt", "w") as f:
    for item in mottext_lst:
        f.write("%s\n" % item)

# mot - motText
motcode_dict = {}
for motcode in list(all_users_pref_mot.Mot.unique()):
    motcode_dict[motcode] = list(
        all_users_pref_mot["MotText"][all_users_pref_mot.Mot == motcode].unique()
    )

with open(out_path + "mot_motText.txt", "w") as f:
    for k, v in motcode_dict.items():
        f.write("'%s':'%s', \n" % (k, v))

**6. (b) Extract specific worthwhileness values**

In [ ]:
user_specific_worthwhileness_table = all_users_pref_mot.copy()

In [ ]:
user_specific_worthwhileness_table.columns

In [ ]:
colstorename = {
    "userid": "userid",
    "Mot": "motid",
    "motsFit": "fit",
    "motsProd": "prod",
    "motsRelax": "enjoy",
}

columnstokeep = [col for col in colstorename.keys()]
colstodrop = set(user_specific_worthwhileness_table.columns) - set(columnstokeep)
csv_columns = [col for col in colstorename.values()]

In [ ]:
user_specific_worthwhileness_table.drop(colstodrop, axis=1, inplace=True)
print(user_specific_worthwhileness_table.shape)

In [ ]:
user_specific_worthwhileness_table.rename(columns=colstorename, inplace=True)
print(user_specific_worthwhileness_table.shape)

In [ ]:
user_specific_worthwhileness_table.columns

In [ ]:
colstypes = {
    "userid": "str",
    "motid": "int",
    "fit": "int",
    "prod": "int",
    "enjoy": "int",
}

user_specific_worthwhileness_table = user_specific_worthwhileness_table.astype(
    colstypes
)
user_specific_worthwhileness_table.head(3)

In [ ]:
## save to csv
output_file = "user_specific_worthwhileness_values.csv"
output_path = os.path.join(anon_dataset_path, output_file)

user_specific_worthwhileness_table.to_csv(
    output_path, index=False, header=True, columns=csv_columns
)

**7. Solve language problem and encodings**

In [ ]:
users_df_with_trips.head(3)

In [ ]:
# gender
print(users_df_with_trips["gender"].unique())
print("\n MAKE MAPPING FROM OTHER LANGUAGES TO EN\n")

users_df_with_trips["gender_en"] = users_df_with_trips["gender"].apply(
    lambda x: user_setting_mapping_dict.get(x)
)
print(users_df_with_trips["gender_en"].unique())

# Language
users_df_with_trips["lang"] = users_df_with_trips["lang"].apply(
    lambda x: "eng" if isinstance(x, str) and x.lower() == "english" else x
)
users_df_with_trips["lang"] = users_df_with_trips["lang"].fillna("-")

# Education
print(users_df_with_trips["degree"].unique())
users_df_with_trips["degree_en"] = users_df_with_trips["degree"].apply(
    lambda x: user_setting_mapping_dict.get(x)
)
print("Make mapping...")
print(users_df_with_trips["degree_en"].unique())
# Fill missing values
users_df_with_trips["degree_en"] = users_df_with_trips["degree_en"].fillna("-")


# maritalStatus
# Fill missings
users_df_with_trips[["maritalStatusHousehold"]] = users_df_with_trips[
    ["maritalStatusHousehold"]
].fillna(value="-")
users_df_with_trips["maritalStatusHousehold"] = users_df_with_trips[
    "maritalStatusHousehold"
].apply(lambda x: "-" if x == "" or x == " " else x)

print(users_df_with_trips["maritalStatusHousehold"].unique())
print("Make mapping...")
users_df_with_trips["maritalStatus"] = users_df_with_trips[
    "maritalStatusHousehold"
].apply(lambda x: user_setting_mapping_dict.get(x))
print(users_df_with_trips["maritalStatus"].unique())
users_df_with_trips[["maritalStatus"]] = users_df_with_trips[["maritalStatus"]].fillna(
    value="-"
)


# Number of people in household
users_df_with_trips[["numberPeopleHousehold"]] = users_df_with_trips[
    ["numberPeopleHousehold"]
].fillna(value="-")
users_df_with_trips["numberPeopleHousehold"] = users_df_with_trips[
    "numberPeopleHousehold"
].apply(lambda x: "-" if x == "" or x == " " else x)

# years of residence in country
users_df_with_trips[["yearsOfResidenceHousehold"]] = users_df_with_trips[
    ["yearsOfResidenceHousehold"]
].fillna(value="-")
users_df_with_trips["yearsOfResidenceHousehold"] = users_df_with_trips[
    "yearsOfResidenceHousehold"
].apply(lambda x: "-" if x == "" or x == " " else x)

# Labour status
# Remove missing
users_df_with_trips[["labourStatusHousehold"]] = users_df_with_trips[
    ["labourStatusHousehold"]
].fillna(value="-")
users_df_with_trips[["labourStatusHousehold"]] = users_df_with_trips[
    "labourStatusHousehold"
].apply(lambda x: "-" if x == "" or x == " " else x)

users_df_with_trips["labourStatusHousehold_en"] = users_df_with_trips[
    "labourStatusHousehold"
].apply(lambda x: user_setting_mapping_dict.get(x))
print(users_df_with_trips["labourStatusHousehold"].unique())
print("Make mapping...")
print(users_df_with_trips["labourStatusHousehold_en"].unique())
users_df_with_trips[["labourStatusHousehold_en"]] = users_df_with_trips[
    ["labourStatusHousehold_en"]
].fillna(value="-")


## Age
users_df_with_trips["age"] = users_df_with_trips.apply(
    lambda x: str(x["minAge"]) + "-" + str(x["maxAge"]), axis=1
)
## 16-24
users_df_with_trips["age"] = users_df_with_trips["age"].apply(
    lambda x: "16-24" if x.startswith(("16-", "20-")) else x
)

## 25-49
users_df_with_trips["age"] = users_df_with_trips["age"].apply(
    lambda x: "25-49" if x.startswith(("25-", "30-", "36-", "40-")) else x
)

## 50-64
users_df_with_trips["age"] = users_df_with_trips["age"].apply(
    lambda x: "50-64" if x.startswith("50-") else x
)

## 65+
users_df_with_trips["age"] = users_df_with_trips["age"].apply(
    lambda x: "65+" if x.startswith(("65-", "75-")) else x
)

In [ ]:
### Campaigns

campaigns_path = os.path.join(meta_data_path, "campaignsInfo.csv")
campaigns = pd.read_csv(campaigns_path)
# create a mapping campaign_id - country
campaignid_country_dict = pd.Series(
    campaigns["country"].values, index=campaigns["campaignId"]
).to_dict()

#################
# users_df_with_trips = pd.read_pickle(input_path + 'users_df_with_trips.pkl')


# there are cases with '' in the list -> ex. ['5cbddfecb7d8af05386ec66e', '']
# there is a user with 3 campaign ids not encoded.
# they are '5b8fb4004dd33478cd30a8b1', '5b27f92c49ee1d1742cacc75', '5b3b4e5cb3863f55591ec50d'
# and appear only once
wrong_codes = [
    "",
    "5b8fb4004dd33478cd30a8b1",
    "5b27f92c49ee1d1742cacc75",
    "5b3b4e5cb3863f55591ec50d",
]

user_with_1camp = 0
user_with_more_camp = 0
user_with_more_camp_same_country = 0

users_df_with_trips["onCampaigns_cor"] = users_df_with_trips["onCampaigns"]
for idx, row in users_df_with_trips.iterrows():
    lst = row["onCampaigns"]

    # user with single campaign
    if len(lst) == 1:
        users_df_with_trips.loc[idx, "onCampaigns_cor"] = campaignid_country_dict[
            lst[0]
        ]
        user_with_1camp += 1

    # user with more campaigns
    if len(lst) > 1:

        new_lst = [
            campaignid_country_dict[lst[i]]
            for i in range(len(lst))
            if lst[i] not in wrong_codes
        ]

        # select unique countries
        new_lst_nodupl = list(dict.fromkeys(new_lst))

        # users with more campaigns but same country
        if len(new_lst_nodupl) == 1:
            user_with_more_camp_same_country += 1
            users_df_with_trips.loc[idx, "onCampaigns_cor"] = new_lst_nodupl[0]

        # users with more campaigns and different countries
        else:
            user_with_more_camp += 1
            # print(new_lst_nodupl)

            # assign these user to 'AAA'
            users_df_with_trips.loc[idx, "onCampaigns_cor"] = "AAA"


print("tot users:", users_df_with_trips.userid.nunique())
print("users with 1 campaign: ", user_with_1camp)
print("users with more campaigns but same country: ", user_with_more_camp_same_country)
print("users with more campaigns and different countries: ", user_with_more_camp)

In [ ]:
# assign 'CHE' to the class Other (AAA)
users_df_with_trips["onCampaigns_cor"] = users_df_with_trips["onCampaigns_cor"].apply(
    lambda x: "AAA" if x == "CHE" else x
)

In [ ]:
users_df_with_trips["onCampaigns"] = users_df_with_trips["onCampaigns_cor"]

In [ ]:
users_df_with_trips.drop("onCampaigns_cor", inplace=True, axis=1)

In [ ]:
users_df_with_trips.groupby("onCampaigns").size().sort_values(
    ascending=False
).reset_index(name="count")

**8. Save**

In [ ]:
users_df_with_trips.to_pickle(out_path + "users_df_with_trips.pkl")

**9. Export CSV table**

In [ ]:
users_details_table = users_df_with_trips.copy()

In [ ]:
colstorename = {
    "userid": "userid",
    "registerTimestamp": "registration_date",
    "gender_en": "gender",
    "age": "age_range",
    "lang": "lang",
    "city": "city",
    "country": "country",
    "degree_en": "education_level",
    "maritalStatusHousehold": "marital_status_household",
    "numberPeopleHousehold": "number_people_household",
    "labourStatusHousehold_en": "labour_status_household",
    "yearsOfResidenceHousehold": "years_of_residence_household",
}

columnstokeep = [col for col in colstorename.keys()]
colstodrop = set(users_details_table.columns) - set(columnstokeep)
csv_columns = [col for col in colstorename.values()]

In [ ]:
users_details_table.drop(colstodrop, axis=1, inplace=True)
print(users_details_table.shape)

In [ ]:
users_details_table.rename(columns=colstorename, inplace=True)
print(users_details_table.shape)

In [ ]:
users_details_table["registration_date"] = pd.to_datetime(
    users_details_table["registration_date"], unit="ms"
)

In [ ]:
nat = np.datetime64("NaT")


def convert_date(timestamp):
    if not pd.isnull(timestamp):
        return datetime.strftime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
    else:
        return ""


users_details_table["registration_date"] = users_details_table["registration_date"].map(
    convert_date
)

In [ ]:
users_details_table.columns

In [ ]:
users_details_table = users_details_table.replace("-", np.nan)

In [ ]:
colstypes = {
    "userid": "str",
    "registration_date": "str",
    "gender": "str",
    "age_range": "str",
    "lang": "str",
    "city": "str",
    "country": "str",
    "education_level": "str",
    "marital_status_household": "str",
    "number_people_household": "str",
    "labour_status_household": "str",
    "years_of_residence_household": "str",
}

users_details_table.astype(colstypes)
users_details_table.head(3)

In [ ]:
users_details_table.groupby("marital_status_household").size()

In [ ]:
## save to csv
output_file = "user_details.csv"
output_path = os.path.join(anon_dataset_path, output_file)
users_details_table.to_csv(output_path, index=False, header=True, columns=csv_columns)

In [ ]:
print(output_path)

<a id='trips_df'></a>
### trips_df

1. Read all data until 30-10-2019
2. Read updated data until 16-12-2019
3. Merge and create `trips_df`
4. Take only trips also in `trips_users_df`
5. Preprocessing
6. Save
7. Extract objectives

**1. All trip data until 30-10-2019**

In [ ]:
# 1. All trips until 30-10-2019

trips_prev_df = []
first = True
read_files = glob.glob(input_path + "*_tripsData.json")
print(read_files)

tot = 0
for f in read_files:
    print(f)
    with open(f) as f:
        trips = json.loads(f.read())
        trips_prev_df_temp = json_normalize(trips)
    print(trips_prev_df_temp.shape)
    tot += trips_prev_df_temp.shape[0]

    if first:
        trips_prev_df = trips_prev_df_temp
        first = False
    else:
        trips_prev_df = pd.concat([trips_prev_df, trips_prev_df_temp])

print()
print("Tot = ", tot)
print(trips_prev_df.shape)
print()
print("Remove duplicates...")
trips_prev_df_nodup = trips_prev_df.drop_duplicates(["tripid"], keep="first")
print(trips_prev_df_nodup.shape)

**2. Updated trips**

In [ ]:
## 2. Updated trips

trips_updated_df = []
first = True
read_files = glob.glob(update_path + "*_tripsData.json")
print(read_files)

tot = 0
for f in read_files:
    print(f)
    with open(f) as f:
        trips = json.loads(f.read())
        trips_updated_df_temp = json_normalize(trips)
    print(trips_updated_df_temp.shape)
    tot += trips_updated_df_temp.shape[0]

    if first:
        trips_updated_df = trips_updated_df_temp
        first = False
    else:
        trips_updated_df = pd.concat(
            [trips_updated_df, trips_updated_df_temp], sort=True
        )

print()
print("Tot = ", tot)
print(trips_updated_df.shape)
print()
print("Remove duplicates...")
trips_updated_df_nodup = trips_updated_df.drop_duplicates(["tripid"], keep="first")
print(trips_updated_df_nodup.shape)

In [ ]:
## check
print("trips DF shape: {}".format(trips_prev_df_nodup.shape))
print("trips_update DF shape: {}".format(trips_updated_df_nodup.shape))

**3. Merge all + updated**

In [ ]:
## 3. Merge all + updated
trips_df_all = pd.concat([trips_prev_df_nodup, trips_updated_df_nodup], sort=True)
print("trips DF shape: {}".format(trips_df_all.shape))

trips_df_all["tripStartDate_formated"] = pd.to_datetime(
    trips_df_all["tripStartDate"], unit="ms"
)
trips_df_all["tripEndDate_formated"] = pd.to_datetime(
    trips_df_all["tripEndDate"], unit="ms"
)

# print('Remove duplicates...')
# trips_df_nodup = trips_df.iloc[trips_df.astype(str).drop_duplicates(keep='first').index]
# print('trips DF shape: {}'.format(trips_df.shape))
print()
print("Total trips: ", trips_df_all.shape[0])
print("Total unique trips: ", len(trips_df_all["tripid"].unique()))

print()
print("==> There are ", trips_df_all.shape[0], "total Trips")
trips_df = trips_df_all[trips_df_all["tripStartDate_formated"] >= CUTTING_DATE]
print("==> There are ", trips_df.shape[0], "Trips after", CUTTING_DATE)

# trips_df.head(3)

**4. Take only trips also in trips_users_df**

In [ ]:
# take only trips in trips_users_df.
# Filtering out trips before the cutting_date
trips_df = trips_df[trips_df.tripid.isin(list(trips_users_df.tripid.unique()))]
print("trips also in trips_users_df", trips_df.shape)

**5. Preprocessing on Dates**

In [ ]:
def getDuration(d1, d2):

    fmt = "%Y-%m-%d %H:%M:%S"
    d1 = datetime.strptime(str(d1)[0:19], fmt)
    d2 = datetime.strptime(str(d2)[0:19], fmt)
    duration_in_s = (d2 - d1).total_seconds()
    #     minutes = divmod(duration_in_s, 60)[0]
    minutes = divmod(duration_in_s, 60)
    #     minutes = round(duration_in_s/60,2)
    return minutes[0] + minutes[1] / 100


trips_df["duration_formated_min"] = trips_df.apply(
    lambda x: getDuration(x["tripStartDate_formated"], x["tripEndDate_formated"]),
    axis=1,
)
# trip start day
trips_df["tripStartDay"] = trips_df["tripStartDate_formated"].dt.date

# Remove trips with duration <=1
trips_df = trips_df[trips_df["duration_formated_min"] > 1]
print(trips_df.shape)

**6. Save**

In [ ]:
## SAVE
trips_df.to_pickle(out_path + "trips_df_original.pkl")

**7. Extract objectives**

In [ ]:
trip_objs = []
first = True
for ix, ob in trips_df.iterrows():
    objs = json_normalize([x for x in ob["objectives"] if x is not None])
    objs["tripid"] = ob["tripid"]
    objs["tripStartDate_formated"] = ob["tripStartDate_formated"]
    if first:
        trip_objs = objs
        first = False
    else:
        trip_objs = pd.concat([trip_objs, objs], sort=True)

print("Initial shape:", trip_objs.shape)

# Convert names in English
trip_obj_dict = {
    "0": "Home",
    "1": "Work",
    "2": "School_Education",
    "3": "Everyday_Shopping",
    "4": "Business_Trip",
    "5": "Leisure_Hobby",
    "6": "Pick_Up_Drop_Off",
    "7": "Personal_Tasks_Errands",
    "8": "Trip_Itself",
    "9": "Other",
}

trip_objs["tripObjectiveStringEN"] = trip_objs["tripObjectiveCode"].apply(
    lambda x: trip_obj_dict[str(int(x))]
)
trip_objs.drop(["tripObjectiveString"], 1, inplace=True)

# Merge 'School_Education' and 'Work'
trip_objs["tripObjectiveStringEN"] = trip_objs["tripObjectiveStringEN"].apply(
    lambda x: "Work" if x == "School_Education" else x
)
# Change 'Trip_Itself' into 'Other'
trip_objs["tripObjectiveStringEN"] = trip_objs["tripObjectiveStringEN"].apply(
    lambda x: "Other" if x == "Trip_Itself" else x
)

print(
    "We know the Trip purpose of ",
    len(trip_objs["tripid"].unique()),
    " trips out of ",
    trips_df.shape[0],
)
trip_objs.head()

In [ ]:
## SAVE
trip_objs.to_pickle(out_path + "trip_objs.pkl")

**Group some objectives as agreed**

- work (includes work and bisiness trip)
- home
- leisure (leisure and hobby)
- shopping
- education (school and education)
- other (the rest)

In [ ]:
# work: work + business_trip
trip_objs["tripObjectiveStringEN"] = trip_objs["tripObjectiveStringEN"].apply(
    lambda x: "Work" if x == "Business_Trip" else x
)
# other
other_lst = ["Personal_Tasks_Errands", "Pick_Up_Drop_Off"]
trip_objs["tripObjectiveStringEN"] = trip_objs["tripObjectiveStringEN"].apply(
    lambda x: "Other" if x in other_lst else x
)

# rename column
trip_objs.rename(columns={"tripObjectiveStringEN": "objective_str"}, inplace=True)


# save
trip_objs.to_pickle(out_path + "trip_objs_grouped.pkl")

<a id='all_legs'></a>
### all_legs

1. Extract legs from `trips_df`
2. Preprocessing on legs
3. Add category transport
4. Add leg duration and preprocessing on duration and distance

**1. Extract legs**

In [ ]:
all_legs_original = []
first = True
for ix, trip in trips_df.iterrows():
    legs = json_normalize([x for x in trip["legs"] if x is not None])
    legs["tripid"] = trip["tripid"]
    legs["tripStartDate_formated"] = trip["tripStartDate_formated"]
    if first:
        all_legs_original = legs
        first = False
    else:
        all_legs_original = pd.concat([all_legs_original, legs])
print(all_legs_original.shape)

In [ ]:
### SAVE original
all_legs_original.to_pickle(out_path + "all_legs_original.pkl")

**2. Preprocessing on legs**

In [ ]:
all_legs = pd.read_pickle(out_path + "all_legs_original.pkl")

In [ ]:
print("Initial shape", all_legs.shape)
all_legs = all_legs.drop_duplicates(["tripid", "legid"], keep="first")
print()
print("Remove rows with same tripid-legid:", all_legs.shape)

# Format date
all_legs["startDate_formated"] = pd.to_datetime(all_legs["startDate"], unit="ms")
all_legs["endDate_formated"] = pd.to_datetime(all_legs["endDate"], unit="ms")

# Filter out legs performed before the cutting date
all_legs = all_legs[all_legs["tripStartDate_formated"] >= CUTTING_DATE]
print()
print("Total legs after the cutting date: ", all_legs.shape[0])

# As suggested by Luis remove columns
# RelaxingFactor
# RelaxingFactorsText
# ProductiveFactorsText
# ProductiveFactors
print()
all_legs = all_legs.drop(
    [
        "ProductiveFactors",
        "ProductiveFactorsText",
        "RelaxingFactors",
        "RelaxingFactorsText",
    ],
    1,
)
print("Remove 4 columns, Luis suggestion..", all_legs.shape)

# Remove legs with correctedModeOfTransport == -1
all_legs = all_legs[all_legs["correctedModeOfTransport"] != -1]
print()
print("Remove legs with correctedModeOfTransport == -1", all_legs.shape)

In [ ]:
# Check on average speed.

transp_mode_dict = {
    "0": {"name": "vehicle", "max_speed": 200},
    "1": {"name": "bicycle", "max_speed": 100},
    "2": {"name": "onfoot", "max_speed": 12},
    "3": {"name": "still", "max_speed": 0},
    "4": {"name": "unknown", "max_speed": 100},
    "5": {"name": "tilting", "max_speed": 100},
    "6": {"name": "inexistent", "max_speed": 100},
    "7": {"name": "walking", "max_speed": 12},
    "8": {"name": "running", "max_speed": 20},
    "9": {"name": "car driver", "max_speed": 250},
    "10": {"name": "train", "max_speed": 350},
    "11": {"name": "tram", "max_speed": 100},
    "12": {"name": "subway", "max_speed": 100},
    "13": {"name": "ferry", "max_speed": 200},
    "14": {"name": "plane", "max_speed": 7000},
    "15": {"name": "bus", "max_speed": 150},
    "16": {"name": "electricBike", "max_speed": 50},
    "17": {"name": "bikeSharing", "max_speed": 50},
    "18": {"name": "microScooter", "max_speed": 50},
    "19": {"name": "skate", "max_speed": 20},
    "20": {"name": "motorcycle", "max_speed": 300},
    "21": {"name": "moped", "max_speed": 80},
    "22": {"name": "carPassenger", "max_speed": 250},
    "23": {"name": "taxi", "max_speed": 250},
    "24": {"name": "rideHailing", "max_speed": 100},
    "25": {"name": "carSharing", "max_speed": 250},
    "26": {"name": "carpooling", "max_speed": 250},
    "27": {"name": "busLongDistance", "max_speed": 150},
    "28": {"name": "highSpeedTrain", "max_speed": 350},
    "29": {"name": "other", "max_speed": 100},
    "30": {"name": "otherPublic", "max_speed": 300},
    "31": {"name": "otherActive", "max_speed": 30},
    "32": {"name": "otherPrivate", "max_speed": 250},
    "33": {"name": "intercityTrain", "max_speed": 300},
    "34": {"name": "wheelChair", "max_speed": 10},
    "35": {"name": "cargoBike", "max_speed": 30},
    "36": {"name": "carSharingPassenger", "max_speed": 250},
    "37": {"name": "electricWheelchair", "max_speed": 30},
}

# put 4 as the code for unknown transport mode
all_legs[["correctedModeOfTransport"]] = all_legs[["correctedModeOfTransport"]].fillna(
    value=4
)
# check the avg speed, True if < than the max_speed (?)
all_legs["check_avg_speed"] = all_legs.apply(
    lambda x: x["averageSpeed"]
    <= transp_mode_dict[str(int(x["correctedModeOfTransport"]))]["max_speed"],
    axis=1,
)
print(all_legs.shape)

**3. Add transport category**

In [ ]:
## Mode of transport dict
trasp_mode = pd.read_csv(meta_data_path + "transport_mode.csv", sep=";")
trasp_mode_dict = trasp_mode.set_index("transport_code").to_dict()["transport_str"]

# create transport category according to spreadsheet
category_transp_mode_dict = {
    "walking": [2, 7, 8, 34, 37],
    "cycling_emerging_micromobility": [1, 16, 17, 18, 19, 31, 35],
    "public_transp_short_dist": [10, 11, 12, 15, 30],
    "public_transp_long_dist": [14, 13, 28, 33, 27],
    "private_motorized": [0, 9, 20, 21, 22, 23, 25, 26, 32, 36],
}
#'Other_Private_Motorised':[32, 21, 20],
#'private_motorized_driver':[0,9,25,26],
#'private_motorized_passenger':[22, 23, 36]}

# save category_transp_mode_dict
with open(out_path + "category_transp_mode_dict.json", "w") as f:
    json.dump(category_transp_mode_dict, f)

# inverted category transp
inverted_category_transp_mode_dict = dict(
    (v, k) for k in category_transp_mode_dict for v in category_transp_mode_dict[k]
)
all_legs["transp_category"] = all_legs.apply(
    lambda x: inverted_category_transp_mode_dict.get(
        int(x["correctedModeOfTransport"])
    ),
    axis=1,
)
all_legs["correctedModeOfTransport_str"] = all_legs["correctedModeOfTransport"].apply(
    lambda x: trasp_mode_dict[x]
)

# Remove transport category 'still'
all_legs = all_legs[all_legs["correctedModeOfTransport_str"] != "still"]

**4. Add leg duration and preprocessing on duration**

In [ ]:
# compute leg duration (in minutes)
print("Compute leg duration in minutes")
print()
all_legs["legDuration"] = all_legs["legDuration"].fillna(0)
all_legs["legDuration_formated_min"] = all_legs["legDuration"].apply(lambda x: x / 60)


def getDuration(d1, d2):

    fmt = "%Y-%m-%d %H:%M:%S"
    d1 = datetime.strptime(str(d1)[0:19], fmt)
    d2 = datetime.strptime(str(d2)[0:19], fmt)
    duration_in_s = (d2 - d1).total_seconds()
    #     minutes = divmod(duration_in_s, 60)[0]
    minutes = divmod(duration_in_s, 60)
    #     minutes = round(duration_in_s/60,2)
    return minutes[0] + minutes[1] / 100


all_legs["inferred_leg_duration_min"] = all_legs.apply(
    lambda x: getDuration(x["startDate_formated"], x["endDate_formated"]), axis=1
)
all_legs["legStartDay"] = pd.to_datetime(all_legs["startDate_formated"]).dt.date

# Remove legs with duration <= 1
all_legs = all_legs[((all_legs["inferred_leg_duration_min"] > 1))]
print("Remove legs with duration <= 1", all_legs.shape)

# Remove legs with distance > 1M meters (1000km) with transport != train
all_legs = all_legs[
    (all_legs["correctedModeOfTransport_str"] != "train")
    | (all_legs["legDistance"] < 1000000)
]
print("Remove legs with distance > 1M meters (1000km)", all_legs.shape)

# Remove legs that last more than 2 days (?)
# ex. all_legs[all_legs.legid == '#22:6670']
print(
    "Trips that last more than 2 days:",
    len(all_legs[all_legs["inferred_leg_duration_min"] > 60 * 48]),
)

# Remove cases in which the same trip is stored with different id

all_legs.head(3)

In [ ]:
## SAVE
all_legs.to_pickle(out_path + "all_legs.pkl")

<a id='all_legs_final_ds_user_info'></a>
### all_legs_final_ds_user_info

1. Merge legs info with users info and trip info

In [ ]:
print("Number of trips: ", len(all_legs["tripid"].unique()))
print("Number of legs: ", len(all_legs["legid"].unique()))

In [ ]:
# trips_users_df: link between users and trips
# users_df_with_trips: demographic info of users

# add to each trip the demographic info of users --> trips_users_df_temp_1
trips_users_df_temp = trips_users_df[["tripid", "userid"]]
trips_users_df_temp_1 = pd.merge(
    trips_users_df_temp, users_df_with_trips, on="userid", how="left"
)

# merge legs and trips+user
all_legs_final_ds_user_info = pd.merge(
    all_legs, trips_users_df_temp_1, on="tripid", how="left"
)

# add info day
all_legs_final_ds_user_info["weekday"] = all_legs_final_ds_user_info[
    "legStartDay"
].apply(lambda x: calendar.day_name[x.weekday()])

we_vs_wd = {"weekend": ["Saturday", "Sunday"]}
all_legs_final_ds_user_info["we_vs_wd"] = all_legs_final_ds_user_info.apply(
    lambda x: "Weekend" if x["weekday"] in we_vs_wd["weekend"] else "Working_day",
    axis=1,
)

In [ ]:
# allcols = list(all_legs_final_ds_user_info.columns)
colstoremove = [
    "legid",
    "tripid",
    "onCampaigns",
    "registerTimestamp",
    "userSettings.actValue",
    "userSettings.city",
    "userSettings.country",
    "userSettings.degree",
    "userSettings.gender",
    "userSettings.hasSetMobilityGoal",
    "userSettings.labourStatusHousehold",
    "userSettings.lang",
    "userSettings.maritalStatusHousehold",
    "userSettings.maxAge",
    "userSettings.minAge",
    "userSettings.mobilityGoalChosen",
    "userSettings.mobilityGoalPoints",
    "userSettings.numberPeopleHousehold",
    "userSettings.pointsPerCampaign",
    "userSettings.preferedMots",
    "userSettings.prodValue",
    "userSettings.relValue",
    "userSettings.yearsOfResidenceHousehold",
    "userSettings.gender_en",
    "userSettings.degree_en",
    "maritalStatus",
    "userSettings.labourStatusHousehold_en",
]
# colsfordup = list(set(allcols) - set(colstoremove))
# cols
# print(all_legs_final_ds_user_info.shape)
# all_legs_final_ds_user_info = all_legs_final_ds_user_info[colsfordup]
# print(all_legs_final_ds_user_info.shape)

In [ ]:
print(all_legs_final_ds_user_info.shape)
all_legs_final_ds_user_info.columns

In [ ]:
## SAVE
all_legs_final_ds_user_info.to_pickle(out_path + "all_legs_final_ds_user_info.pkl")

<a id='all_legs_merged'></a>
### all_legs_merged

Merge legs that belong to the same user and have the same mode of transport and the time between the two legs is less than 5 minutes.
<br>Merged Variables:
- end date: the end date formatted of the second leg
- max speed: take the maximum of the two speeds
- trueDistance: take the sum of the two distances
- inferred_leg_duration_min: take the sum of the two distances
- wastedTime: consider as wasted time the average of the WT of merged legs
- wastedTime_withtime: compute also the worthwhileness in relation to travelled time.


1. Merge legs
2. Average Wasted Time for merged legs

**1. Merge legs**

In [ ]:
all_legs_final_ds_user_info = pd.read_pickle(
    out_path + "all_legs_final_ds_user_info.pkl"
)
print(all_legs_final_ds_user_info.shape)
print(
    "Number of users with a leg: ", len(all_legs_final_ds_user_info["userid"].unique())
)
print(
    "Legs with duration < 5:",
    len(
        all_legs_final_ds_user_info[
            ((all_legs_final_ds_user_info["inferred_leg_duration_min"] < 1))
        ]
    ),
)
print(
    "Legs with class different from Leg:",
    len(all_legs_final_ds_user_info[all_legs_final_ds_user_info["class"] != "Leg"]),
)

In [ ]:
all_legs_final_ds_user_info = pd.read_pickle(
    out_path + "all_legs_final_ds_user_info.pkl"
)
print(all_legs_final_ds_user_info.shape)
print(
    "Number of users with a leg: ", len(all_legs_final_ds_user_info["userid"].unique())
)
print(
    "Legs with duration < 5:",
    len(
        all_legs_final_ds_user_info[
            ((all_legs_final_ds_user_info["inferred_leg_duration_min"] < 1))
        ]
    ),
)
print(
    "Legs with class different from Leg:",
    len(all_legs_final_ds_user_info[all_legs_final_ds_user_info["class"] != "Leg"]),
)

In [ ]:
all_legs_final_ds_user_info.groupby("class").size()

In [ ]:
def getDuration(d1, d2):
    fmt = "%Y-%m-%d %H:%M:%S"
    d1 = datetime.strptime(str(d1)[0:19], fmt)
    d2 = datetime.strptime(str(d2)[0:19], fmt)
    duration_in_s = (d2 - d1).total_seconds()
    minutes = divmod(duration_in_s, 60)
    return minutes[0] + minutes[1] / 100

In [ ]:
all_legs_for_merging = all_legs_final_ds_user_info.copy()
# dictionary to keep track of merged legs final_legid: [list of merged legs id]
merged_legs = {}
cols = list(all_legs_for_merging.columns)
all_legs_merged = pd.DataFrame(columns=cols)

uix = 1
for user in all_legs_for_merging["userid"].unique():
    # print(uix)
    # user = 'o3QNooVleBR9WfbIaCqsaj3DlDl2'
    uix += 1
    first = True
    curruser_legs = all_legs_for_merging[
        all_legs_for_merging["userid"] == user
    ].sort_values("startDate_formated", ascending=True)

    # if the user has only a leg
    if curruser_legs.shape[0] == 1:
        all_legs_merged = pd.concat([all_legs_merged, pd.DataFrame(curruser_legs)])
    else:
        # for each leg (except the last one) get the consecutive ones
        for leg_ix in range(curruser_legs.shape[0] - 1):
            # print(currleg.index[leg_ix])
            currleg = curruser_legs.iloc[
                leg_ix : leg_ix + 1,
            ]
            currleg_s = curruser_legs.iloc[
                leg_ix,
            ]
            nextleg = curruser_legs.iloc[
                leg_ix + 1 : leg_ix + 2,
            ]
            nextleg_s = curruser_legs.iloc[
                leg_ix + 1,
            ]

            if first:
                # print('first')
                all_legs_merged = pd.concat([all_legs_merged, pd.DataFrame(currleg)])
                first = False
            # if same transport mode
            if (
                curruser_legs.iloc[leg_ix,]["correctedModeOfTransport_str"]
                == curruser_legs.iloc[leg_ix + 1,]["correctedModeOfTransport_str"]
            ):
                # get the duration
                deltatime = getDuration(
                    curruser_legs.iloc[leg_ix + 1,]["startDate_formated"],
                    curruser_legs.iloc[leg_ix,]["endDate_formated"],
                )
                # MERGE if the duration is < 5
                if np.abs(deltatime) < 5:
                    # append to the last position of all_legs_merged dataset the merged leg

                    all_legs_merged.iloc[
                        -1, all_legs_merged.columns.get_loc("endDate_formated")
                    ] = nextleg_s["endDate_formated"]
                    all_legs_merged.iloc[
                        -1, all_legs_merged.columns.get_loc("maxSpeed")
                    ] = max(
                        all_legs_merged.iloc[
                            -1, all_legs_merged.columns.get_loc("maxSpeed")
                        ],
                        nextleg_s["maxSpeed"],
                    )
                    all_legs_merged.iloc[
                        -1, all_legs_merged.columns.get_loc("trueDistance")
                    ] = (
                        all_legs_merged.iloc[
                            -1, all_legs_merged.columns.get_loc("trueDistance")
                        ]
                        + nextleg_s["trueDistance"]
                    )
                    all_legs_merged.iloc[
                        -1, all_legs_merged.columns.get_loc("inferred_leg_duration_min")
                    ] = (
                        all_legs_merged.iloc[
                            -1,
                            all_legs_merged.columns.get_loc(
                                "inferred_leg_duration_min"
                            ),
                        ]
                        + nextleg_s["inferred_leg_duration_min"]
                    )

                    # create the dictionary
                    try:
                        merged_legs[
                            all_legs_merged.iloc[
                                -1, all_legs_merged.columns.get_loc("legid")
                            ]
                        ].append(nextleg_s["legid"])
                    except:
                        merged_legs[
                            all_legs_merged.iloc[
                                -1, all_legs_merged.columns.get_loc("legid")
                            ]
                        ] = []
                        merged_legs[
                            all_legs_merged.iloc[
                                -1, all_legs_merged.columns.get_loc("legid")
                            ]
                        ].append(nextleg_s["legid"])

                else:  # if duration > 5

                    all_legs_merged = pd.concat(
                        [all_legs_merged, pd.DataFrame(nextleg)]
                    )

            else:  # if different transport mode
                all_legs_merged = pd.concat([all_legs_merged, pd.DataFrame(nextleg)])

In [ ]:
all_legs_merged.shape

In [ ]:
### SAVE
all_legs_merged.to_pickle(out_path + "all_legs_merged.pkl")
with open(out_path + "merged_legs.json", "w") as file:
    file.write(json.dumps(merged_legs))

**2. Average Wasted Time for merged legs**

For the merged legs add the columns:

- `wastedTime`: consider as wasted time the average of the WT of merged legs
- `wastedTime_withtime`: compute also the worthwhileness in relation to travelled time. Given 2 legs l1 and l2, to obtain the related wasted times we compute $((l1*t1)+(l2*t2))/(t1+t2)$

In [ ]:
# Read data
all_legs_merged = pd.read_pickle(out_path + "all_legs_merged.pkl")

with open(out_path + "merged_legs.json") as json_file:
    merged_legs = json.load(json_file)

print("Merged legs:", len(all_legs_merged))
print("Different trips:", len(all_legs_merged.tripid.unique()))
print("Users:", len(all_legs_merged.userid.unique()))

In [ ]:
all_legs_merged["tot_wastedTime"] = 0
all_legs_merged["count_wastedTime"] = 0
all_legs_merged["wastedTime_withtime"] = all_legs_merged["wastedTime"]

tot_w = 0
count_w = 0

tot_speed = 0
count_speed = 0

tot_wt = 0
count_wt = 0

for k_legid, v_mergedlegs in merged_legs.items():
    # print(k_legid)
    ## Merged leg
    # take the average speed
    tot_speed = all_legs_final_ds_user_info[
        all_legs_final_ds_user_info["legid"] == k_legid
    ].T.squeeze()["averageSpeed"]
    count_speed = 1
    # take the WT
    curr_w = all_legs_final_ds_user_info[
        all_legs_final_ds_user_info["legid"] == k_legid
    ].T.squeeze()["wastedTime"]
    # take the duration of current leg
    curr_time = all_legs_final_ds_user_info[
        all_legs_final_ds_user_info["legid"] == k_legid
    ].T.squeeze()["inferred_leg_duration_min"]

    if curr_w >= 1 and curr_w <= 5:
        tot_w = curr_w
        count_w = 1

        # we want to compute also the worthwhilness in relation to travelled time
        # so given 2 legs l1 and l2 we compute ((l1*t1)+(l2*t2))/t1+t2
        tot_wt = curr_w * curr_time
        count_wt = curr_time

    for l in v_mergedlegs:

        tot_speed += all_legs_final_ds_user_info[
            all_legs_final_ds_user_info["legid"] == l
        ].T.squeeze()["averageSpeed"]
        count_speed += 1

        curr_w = all_legs_final_ds_user_info[
            all_legs_final_ds_user_info["legid"] == l
        ].T.squeeze()["wastedTime"]
        curr_time = all_legs_final_ds_user_info[
            all_legs_final_ds_user_info["legid"] == l
        ].T.squeeze()["inferred_leg_duration_min"]
        if curr_w >= 1 and curr_w <= 5:
            tot_w += curr_w
            count_w += 1
            # we want to compute Also the worthwhilness in relation to travelled time
            # so given 2 legs l1 and l2 we compute ((l1*t1)+(l2*t2))/t1+t2
            tot_wt += curr_w * curr_time
            count_wt += curr_time

    all_legs_merged.loc[all_legs_merged["legid"] == k_legid, ["wastedTime"]] = (
        tot_w / count_w
    )  # average WT of merged legs
    all_legs_merged.loc[all_legs_merged["legid"] == k_legid, ["tot_wastedTime"]] = tot_w
    all_legs_merged.loc[
        all_legs_merged["legid"] == k_legid, ["count_wastedTime"]
    ] = count_w

    try:
        all_legs_merged.loc[
            all_legs_merged["legid"] == k_legid, ["wastedTime_withtime"]
        ] = (tot_wt / count_wt)
    except:
        print(v_mergedlegs)
        break

    all_legs_merged.loc[all_legs_merged["legid"] == k_legid, ["averageSpeed"]] = (
        tot_speed / count_speed
    )

all_legs_merged.to_pickle(out_path + "all_legs_merged_1.pkl")

In [ ]:
all_legs_merged.shape

<a id='all_factors'></a>
### all_factors

For each leg there are some factors that can have positive or negative (or both) influence.
They can be divided into:
- **GT:** gettingThereFactors
- **ACT:** activitiesFactors
- **WYR:** whileYouRideFactors
- **CP:** comfortAndPleasentFactors

-----------

1. Select factors from each leg
2. Select generic activities from each leg
3. Create the Encodings
4. Map the encoded factors
5. Check not encoded factors

**1. Select factors from each leg**
<br>**2. Select generic activities from each leg**

In [ ]:
all_factors = []
all_gen_act = []
first = True
i = 0
for index, leg in all_legs_merged.iterrows():

    gt_factors = pd.DataFrame(list(leg["gettingThereFactors"]))
    gt_factors["type"] = "GT"

    act_fact = pd.DataFrame(list(leg["activitiesFactors"]))
    act_fact["type"] = "ACT"
    curr_factors = pd.concat([gt_factors, act_fact])

    wyr_fact = pd.DataFrame(list(leg["whileYouRideFactors"]))
    wyr_fact["type"] = "WYR"
    curr_factors = pd.concat([curr_factors, wyr_fact])

    cp_fact = pd.DataFrame(list(leg["comfortAndPleasentFactors"]))
    cp_fact["type"] = "CP"
    curr_factors = pd.concat([curr_factors, cp_fact])

    curr_factors["tripid"] = leg["tripid"]
    curr_factors["legid"] = leg["legid"]
    curr_factors["correctedModeOfTransport"] = leg["correctedModeOfTransport"]
    curr_factors["correctedModeOfTransport_str"] = leg["correctedModeOfTransport_str"]
    curr_factors["legStartDay"] = leg["legStartDay"]

    ### ?
    gen_act = pd.DataFrame(list(leg["genericActivities"]))
    gen_act["tripid"] = leg["tripid"]
    gen_act["legid"] = leg["legid"]
    gen_act["legStartDay"] = leg["legStartDay"]

    if first:
        all_factors = curr_factors
        all_gen_act = gen_act
        first = False
    else:
        all_factors = pd.concat([all_factors, curr_factors])
        all_gen_act = pd.concat([all_gen_act, gen_act])


all_factors.reset_index(inplace=True)
all_factors.drop("index", 1)
print("Number of legs containing factors", len(all_factors["legid"].unique()))
print("Remember: a leg may contain multiple factors\n")
print("All factors:", all_factors.shape)
print()
all_gen_act = all_gen_act.reset_index()
all_gen_act = all_gen_act.drop("text", 1)
print("all_gen_act:", all_gen_act.shape)

In [ ]:
## SAVE
all_factors.to_pickle(out_path + "all_factors_original.pkl")
# all_gen_act.to_pickle(out_path + "all_gen_act.pkl")

**2. Select generic activities from each leg**

- If the activity is driving and the TC is walking, change into **walking**.
- If the activity is driving and the TC is cycling, change into **cycling**.
- If the activity is driving and the TC is public transport, **drop the leg**.

In [ ]:
# Read data
all_legs_merged = pd.read_pickle(out_path + "all_legs_merged_1.pkl")

all_gen_act = pd.read_pickle(out_path + "all_gen_act.pkl")
# rename values in  column 'code'
all_gen_act["code"] = all_gen_act["code"].apply(lambda x: x[10:])

# add transp_category
all_gen_act = all_gen_act.merge(
    all_legs_merged[["legid", "transp_category"]], on="legid"
).drop_duplicates()

# convert walking and cycling
def convert_driving(row):
    if row["transp_category"] == "walking" and row["code"] == "Driving":
        return "Walking"
    if (
        row["transp_category"] == "cycling_emerging_micromobility"
        and row["code"] == "Driving"
    ):
        return "Cycling"
    else:
        return row["code"]


all_gen_act["code"] = all_gen_act.apply(lambda row: convert_driving(row), axis=1)

# drop if public transport -- 6 legs!!
xx = all_gen_act[
    (
        all_gen_act.transp_category.isin(
            ["public_transp_short_dist", "public_transp_long_dist"]
        )
    )
    & (all_gen_act.code == "Driving")
]
print("Legs with driving and public transport: ", len(xx))
print(all_gen_act.shape)
all_gen_act = all_gen_act[
    ~(
        (
            all_gen_act.transp_category.isin(
                ["public_transp_short_dist", "public_transp_long_dist"]
            )
        )
        & (all_gen_act.code == "Driving")
    )
]
print(all_gen_act.shape)
all_gen_act.head()

In [ ]:
## SAVE
all_gen_act.to_pickle(out_path + "all_gen_act.pkl")

**3. Create the Encodings**

There are two different encodings:

- the one for the codes from 0 to 11 depending on the transport category and on the type of activity
- the one for the cosed from 1001 to 3212

We created also the encodings for the codes that are actually strings.

**oss:** in `all_factors` there are also codes with strings that need to be encoded/preprocessed
<br> **oss:** in the encoding files there is a mistake. `Crowdness_eating` becomes `Crowdness_Seating`
<br>**oss:** in the encoding files the transport category 'carDriver' is 'car'.


In [ ]:
all_factors = pd.read_pickle(out_path + "all_factors_original.pkl")
all_factors.drop("index", 1, inplace=True)
all_factors.reset_index(inplace=True)
all_factors.drop("index", 1, inplace=True)
print("All factors shape:", all_factors.shape)
all_factors.head()

In [ ]:
all_factors.groupby("code").size()

In [ ]:
# For each type of factors read the csv with the encoding


##GettingThereFactors
activeTranspGTFactors = pd.read_csv(
    meta_data_path + "activeTranspGTFactors.csv", sep=";"
)
privateTransportGTFactors = pd.read_csv(
    meta_data_path + "PrivateTransportGTFactors.csv", sep=";"
)
publicTransportGTFactors = pd.read_csv(
    meta_data_path + "PublicTransportGTFactors.csv", sep=";"
)

GTFactors = pd.concat(
    [activeTranspGTFactors, privateTransportGTFactors, publicTransportGTFactors]
)
GTFactors["mode_activity"] = GTFactors["mode_activity"].apply(
    lambda x: "carDriver" if x == "car" else x
)
GTFactors["factor_k"] = GTFactors["factor_k"].apply(
    lambda x: "Crowdedness_Seating" if x == "Crowdedness_eating" else x
)

# for each transport category create a dictionary with code: factors
# transport category : {code: factors}
gt_fact = {}
for index, fac in GTFactors.iterrows():
    try:
        gt_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]
    except:
        gt_fact[fac["mode_activity"]] = {}
        gt_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]


# ============================================
##ACTFactors
activeTransportACTFactors = pd.read_csv(
    meta_data_path + "ActiveTransportACTFactors.csv", sep=";"
)
privateTransportACTFactors = pd.read_csv(
    meta_data_path + "PrivateTransportACTFactors.csv", sep=";"
)
publicTransportACTFactors = pd.read_csv(
    meta_data_path + "PublicTransportACTFactors.csv", sep=";"
)

ACT_factors = pd.concat(
    [activeTransportACTFactors, privateTransportACTFactors, publicTransportACTFactors]
)
ACT_factors["mode_activity"] = ACT_factors["mode_activity"].apply(
    lambda x: "carDriver" if x == "car" else x
)
ACT_factors["factor_k"] = ACT_factors["factor_k"].apply(
    lambda x: "Crowdedness_Seating" if x == "Crowdedness_eating" else x
)

act_fact = {}
for index, fac in ACT_factors.iterrows():
    try:
        act_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]
    except:
        act_fact[fac["mode_activity"]] = {}
        act_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]


# ============================================
##WYRFactors
activeTransportWYRFactors = pd.read_csv(
    meta_data_path + "ActiveTransportWYRFactors.csv", sep=";"
)
privateTransportWYRFactors = pd.read_csv(
    meta_data_path + "PrivateTransportWYRFactors.csv", sep=";"
)
publicTransportWYRFactors = pd.read_csv(
    meta_data_path + "PublicTransportWYRFactors.csv", sep=";"
)

WYR_factors = pd.concat(
    [activeTransportWYRFactors, privateTransportWYRFactors, publicTransportWYRFactors]
)
WYR_factors["mode_activity"] = WYR_factors["mode_activity"].apply(lambda x: x.strip())
WYR_factors["mode_activity"] = WYR_factors["mode_activity"].apply(
    lambda x: "carDriver" if x == "car" else x
)
WYR_factors["factor_k"] = WYR_factors["factor_k"].apply(
    lambda x: "Crowdedness_Seating" if x == "Crowdedness_eating" else x
)

wyr_fact = {}
for index, fac in WYR_factors.iterrows():
    try:
        wyr_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]
    except:
        wyr_fact[fac["mode_activity"]] = {}
        wyr_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]


# ============================================
##CPFactors
publicTransportCPFactors = pd.read_csv(
    meta_data_path + "PublicTransportCPFactors.csv", sep=";"
)
privateTransportCPFactors = pd.read_csv(
    meta_data_path + "PrivateTransportCPFactors.csv", sep=";"
)
activeTransportCPFactors = pd.read_csv(
    meta_data_path + "ActiveTransportCPFactors.csv", sep=";"
)

CP_factors = pd.concat(
    [activeTransportCPFactors, publicTransportCPFactors, privateTransportCPFactors]
)
CP_factors["mode_activity"] = CP_factors["mode_activity"].apply(
    lambda x: "carDriver" if x == "car" else x
)
CP_factors["factor_k"] = CP_factors["factor_k"].apply(
    lambda x: "Crowdedness_Seating" if x == "Crowdedness_eating" else x
)

cp_fact = {}
for index, fac in CP_factors.iterrows():
    try:
        cp_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]
    except:
        cp_fact[fac["mode_activity"]] = {}
        cp_fact[fac["mode_activity"]][fac["code"]] = fac["factor_k"]


### Experience Factors
# {type: {transport_category : {code: factor}}}
# {'GT': {'walking': {0: 'Simplicity_Difficulty_Of_The_Route', ...

exp_fact = {}
exp_fact["GT"] = gt_fact
exp_fact["ACT"] = act_fact
exp_fact["WYR"] = wyr_fact
exp_fact["CP"] = cp_fact


### Create a list with all the factors
factor_list = set(GTFactors["factor_k"].unique())
factor_list = factor_list.union(set(ACT_factors["factor_k"].unique()))
factor_list = factor_list.union(set(WYR_factors["factor_k"].unique()))
factor_list = factor_list.union(set(CP_factors["factor_k"].unique()))
print("Total factors:", len(factor_list))
print()


### Strings in the original all_factors.
# Factors are all the same with the exception of
# 'Ability_To_Do_The_Things_I_Want' and 'Space_onboard_for_luggage_pram_bicycle'
str_codes_factors = all_factors[~all_factors["code"].isin(np.arange(0, 5000))]
str_factors_lst = set(str_codes_factors.code.unique())
# change 'Ability_To_Do_The_Things_I_Want' in 'Ability_To_Do_What_I_Wanted'
# change 'Space_onboard_for_luggage_pram_bicycle' in 'Space_Onboard_For_Lugagge_Pram_Bicycle'
all_factors["code"] = all_factors["code"].apply(
    lambda x: "Ability_To_Do_What_I_Wanted"
    if x == "Ability_To_Do_The_Things_I_Want"
    else x
)
all_factors["code"] = all_factors["code"].apply(
    lambda x: "Space_Onboard_For_Lugagge_Pram_Bicycle"
    if x == "Space_onboard_for_luggage_pram_bicycle"
    else x
)


### New encoding
new_fact_encoding = pd.read_csv(
    meta_data_path + "encoding-wip-no-colisions-of-IDs_NEW.csv", sep=";"
)

new_fact = {}
for index, fac in new_fact_encoding.iterrows():
    new_fact[fac["code"]] = fac["new_factor"]

print("Total new factors:", len(set(new_fact_encoding.new_factor.unique())))
new_factors = set(new_fact_encoding.new_factor.unique())
all_factors_lst = new_factors.union(factor_list)

new_fact_encoding.head()

In [ ]:
print("--- Check intersection between encodings ---")

print("Factors only in the first encoding: \n", factor_list.difference(new_factors))
print()
print("Factors only in new encoding: \n", new_factors.difference(factor_list))
print()
print("--- Total number of encoded factors:", len(all_factors_lst))

**4. Map the encoded factors**

In [ ]:
def getFactor(fac_type, transp, code):
    temp_transp_str = trasp_mode_dict[int(transp)]

    # if it is already a string, use it as factor
    if type(code) == str:
        fact_str = code

    else:
        try:  # check if it was in the first encoding
            fact_str = exp_fact[fac_type][temp_transp_str][float(code)]
        except:
            try:  # check if it was in the new encoding
                fact_str = new_fact[float(code)]
            except:
                fact_str = ""
        return fact_str


all_factors["factor"] = all_factors.apply(
    lambda x: getFactor(x["type"], x["correctedModeOfTransport"], x["code"]), axis=1
)
all_factors.head()

**5. Check not encoded factors**

- mode of transport `unknown` and `otherPublic` are not present in the encodings
- we have the encoding for the walking category from 0 to 8 but there is a single leg (#25:5856) with code 9 and 10

In [ ]:
not_encoded_factors = all_factors[all_factors.factor == ""]
print("legs without encoding:", not_encoded_factors.shape)

not_encoded_factors.head()

In [ ]:
### SAVE
# all_factors.to_pickle(out_path + "all_factors.pkl")

**6. Merge factors**

- Air_Quality_Temperature and Air_Quality --> Air_Quality
- Predictability_Of_Travel_Time, Schedule_Reliability,  Reliability_Of_Travel_Time --> Reliability_Of_Travel_Time
- Urban_Scenery_And_Atmosphere, Scenery, Nature_And_Scenery --> Scenery
- Ability_To_Take_Kids_Or_Pets_Along’ , ‘Ability_To_Take_Pets_Along’ --> Ability_To_Take_Kids_Or_Pets_Along
- Accessibility_Escalators_Lifts_Ramps_Stairs_Etc , Convenient_Access_Lifts_Boarding --> Convenient_Access

In [ ]:
all_factors = pd.read_pickle(out_path + "all_factors.pkl")
print("before preprocessing: ", all_factors.factor.nunique())
# air quality
all_factors["factor"] = all_factors["factor"].apply(
    lambda x: "Air_Quaility" if x == "Air_Quality_Temperature" else x
)

# Reliability_Of_Travel_Time
rel_lst = [
    "Predictability_Of_Travel_Time",
    "Schedule_Reliability",
    "Reliability_Of_Travel_Time",
]
all_factors["factor"] = all_factors["factor"].apply(
    lambda x: "Reliability_Of_Travel_Time" if x in rel_lst else x
)

# Scenery
lst = ["Urban_Scenery_And_Atmosphere", "Scenery", "Nature_And_Scenery"]
all_factors["factor"] = all_factors["factor"].apply(
    lambda x: "Scenery" if x in lst else x
)

# Ability_To_Take_Kids_Or_Pets_Along
all_factors["factor"] = all_factors["factor"].apply(
    lambda x: "Ability_To_Take_Kids_Or_Pets_Along"
    if x == "Ability_To_Take_Pets_Along"
    else x
)

# Accessibility
lst = [
    "Accessibility_Escalators_Lifts_Ramps_Stairs_Etc",
    "Convenient_Access_Lifts_Boarding",
]
all_factors["factor"] = all_factors["factor"].apply(
    lambda x: "Convenient_Access" if x in lst else x
)

print("after preprocessing: ", all_factors.factor.nunique())

In [ ]:
### SAVE
all_factors.to_pickle(out_path + "all_factors.pkl")

**7. Raw activities**

In [ ]:
# load data
all_legs_merged = pd.read_pickle(out_path + "all_legs_merged_1.pkl")
print(all_legs_merged.shape)

In [ ]:
raw_act = all_legs_merged[["tripid", "legid", "genericActivities"]].copy()

In [ ]:
def count_act(act_dict_list):
    return len(act_dict_list)

In [ ]:
raw_act["count_act"] = raw_act["genericActivities"].apply(count_act)

In [ ]:
raw_act.groupby("count_act").count()

In [ ]:
raw_act_clean = raw_act.loc[~raw_act["genericActivities"].isnull()]

In [ ]:
raw_act_clean.loc[raw_act_clean["count_act"] == 2].head(5)

In [ ]:
raw_act_clean.head()

In [ ]:
raw_act_clean.loc[raw_act_clean["count_act"] > 3].head(5)

In [ ]:
activities_col = pd.DataFrame(
    raw_act_clean["genericActivities"].values.tolist(),
    index=raw_act_clean.index,
    columns=["activity_{:02d}".format(i) for i in range(1, 10 + 1)],
)

In [ ]:
activities_col.head()

In [ ]:
example_indexes = raw_act_clean.loc[raw_act_clean["count_act"] > 3].head(5).index

In [ ]:
activities_col.loc[example_indexes]

In [ ]:
all_raw_act = raw_act[["tripid", "legid", "count_act"]].join(activities_col)

all_raw_act.head(3)

In [ ]:
all_raw_activities_rows = (
    all_raw_act.set_index(["tripid", "legid", "count_act"])
    .stack()
    .reset_index(name="activity_dict")
    .rename(columns={"level_3": "activity_no"})
)

In [ ]:
def map_act_dict(actdict):
    return actdict["text"], actdict["code"]


(
    all_raw_activities_rows["activity_text"],
    all_raw_activities_rows["activity_code"],
) = zip(*all_raw_activities_rows["activity_dict"].apply(map_act_dict))

all_raw_activities_rows.drop("activity_dict", inplace=True, axis=1)

In [ ]:
all_raw_activities_rows.loc[all_raw_activities_rows["count_act"] > 2].head(10)

In [ ]:
### SAVE
all_raw_activities_rows.to_pickle(out_path + "all_raw_activities_rows.pkl")

In [ ]:
# Export to CSV
activity_labels = (
    all_raw_activities_rows.groupby(["activity_text", "activity_code"])
    .size()
    .to_frame("size")
    .sort_values(by="size", ascending=False)
    .reset_index()
)

activity_labels.to_csv(out_path + "activity_labels.csv", index=False)

<a id='values_from_trip'></a>
### values_from_trip

1. Select values from trips
2. Mapping the value to the Activity (Enjoyment, Fitness, Payed Work, Productivity)

**1. Select values from trips**

In [ ]:
all_values_from_trip = []
first = True
for index, leg in all_legs_merged.iterrows():
    vft_curr = pd.DataFrame(list(leg["valueFromTrip"]))
    vft_curr["tripid"] = leg["tripid"]
    vft_curr["legid"] = leg["legid"]
    vft_curr["legStartDay"] = leg["legStartDay"]

    if first:
        all_values_from_trip = vft_curr
        first = False
    else:
        all_values_from_trip = pd.concat([all_values_from_trip, vft_curr])

print(all_values_from_trip.shape)
all_values_from_trip.head(4)

**2. Mapping the value to the Activity (Enjoyment, Fitness, Payed Work, Productivity)**

In [ ]:
# add the activity
vft_dict = {
    "-1": "Unknown",
    "0": "Paid_work",
    "1": "Personal_tasks",
    "2": "Enjoyment",
    "3": "Fitness",
}
all_values_from_trip["valueFromTrip"] = all_values_from_trip["code"].apply(
    lambda x: vft_dict[str(int(x))]
)

print(
    "Legs with values from trips:",
    len(all_values_from_trip["legid"].unique()),
    " - ",
    np.round(
        len(all_values_from_trip["legid"].unique())
        / len(all_legs["legid"].unique())
        * 100,
        2,
    ),
    "%",
)
print(
    "Trips with a values from trips:",
    len(all_values_from_trip["tripid"].unique()),
    " - ",
    np.round(
        len(all_values_from_trip["tripid"].unique())
        / len(all_legs["tripid"].unique())
        * 100,
        2,
    ),
    "%",
)

all_values_from_trip.head()

In [ ]:
### SAVE
all_values_from_trip.to_pickle(out_path + "values_from_trip.pkl")

<a id='outliers_detection'></a>
### outliers_detection

Check outliers for each mode of transport using the variable **`legDistance`**

1. Create lower and upper bounds
2. Remove outliers

In [ ]:
# load data
all_legs_merged = pd.read_pickle(out_path + "all_legs_merged_1.pkl")
print(all_legs_merged.shape)

In [ ]:
variable = "legDistance"
# sort all the legs according to the duration
legs_in_multi_legs_trips = all_legs_merged.sort_values(
    ["tripid", variable], ascending=[True, False]
)
# order for each trip the legs by duration
legs_in_multi_legs_trips["rank"] = (
    legs_in_multi_legs_trips.groupby(["tripid"]).cumcount() + 1
)
# take the first leg -> the longest
# should be one leg for each trip
longest_legs = legs_in_multi_legs_trips[legs_in_multi_legs_trips["rank"] == 1]
print("Number of longest legs equal to number of trips")
print("trips:", len(legs_in_multi_legs_trips.tripid.unique()))
print("longest legs:", len(longest_legs))
print()

### Use the quantile method
# remove the observations having the lowest and the highest values
quant = 0.01
low_quant = quant
up_quant = 1 - quant

**1. Create lower and upper bounds**

In [ ]:
# create the upper bounds of distance and duration for each mode of transport
stats_uptime = (
    longest_legs.groupby("correctedModeOfTransport_str")[
        "inferred_leg_duration_min", "legDistance"
    ]
    .quantile(up_quant)
    .reset_index()
)
stats_uptime.columns = [
    "correctedModeOfTransport_str",
    "up_bound_time",
    "up_bound_dist",
]

# create the lower bounds of distance and duration for each mode of transport
stats_lowtime = (
    longest_legs.groupby("correctedModeOfTransport_str")[
        "inferred_leg_duration_min", "legDistance"
    ]
    .quantile(low_quant)
    .reset_index()
)
stats_lowtime.columns = [
    "correctedModeOfTransport_str",
    "low_bound_time",
    "low_bound_dist",
]

# merge and add a count
stats = pd.merge(
    stats_lowtime, stats_uptime, on="correctedModeOfTransport_str", how="left"
)
stats = pd.merge(
    stats_lowtime, stats_uptime, on="correctedModeOfTransport_str", how="left"
)
count_stat = longest_legs.groupby("correctedModeOfTransport_str").size().reset_index()
count_stat.columns = ["correctedModeOfTransport_str", "count"]
stats = pd.merge(count_stat, stats, on="correctedModeOfTransport_str", how="left")

stats

**2. Remove outliers**

In [ ]:
all_legs_merged_no_outlier_no_outlier = pd.merge(
    all_legs_merged_no_outlier, stats, on="correctedModeOfTransport_str", how="left"
)

print("Initial number of legs: ", all_legs_merged_no_outlier_no_outlier.shape[0])
all_legs_merged_no_outlier = all_legs_merged_no_outlier[
    (
        (
            all_legs_merged_no_outlier["legDistance"]
            >= all_legs_merged_no_outlier["low_bound_dist"]
        )
        & (
            all_legs_merged_no_outlier["legDistance"]
            <= all_legs_merged_no_outlier["up_bound_dist"]
        )
        & (
            all_legs_merged_no_outlier["inferred_leg_duration_min"]
            >= all_legs_merged_no_outlier["low_bound_time"]
        )
        & (
            all_legs_merged_no_outlier["inferred_leg_duration_min"]
            <= all_legs_merged_no_outlier["up_bound_time"]
        )
    )
    | (all_legs_merged_no_outlier["class"] != "Leg")
]
print("Legs without outliers: ", all_legs_merged_no_outlier.shape[0])

In [ ]:
### SAVE
all_legs_merged_no_outlier.to_pickle(
    out_path + "all_legs_merged_no_outlier_" + str(quant) + ".pkl"
)